<a href="https://colab.research.google.com/github/NourFakih/Vit-gpt2-image-captioning-project/blob/main/Vit_image_captioning%2Bcoco%2BFlickr8k%2B30k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'flickr8k:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F623289%2F1111676%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T095117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0a58179cb13cfb4a6e8a41b05aba43b608e43ab114dab73844766fe088fdd1f2d860c123cb8853919dfcdbc50133f9a60cdf6b021db83b36a6e089d370c44a458b549935da1bcdf4eb7405698ebab1523d33879e4dcda96ee2d77247c083e641bb65127fab4ad23c0adf7021025290599eba5d88e0f9d00ca8170ad8d3eda16a81ba31d359384d779411ee5b34d90787c88226e855c3de1cc6e5b68572cfda93aaecbb769021165b5fb2052808fb55f43546217a58a5b26630c818f4df0bcba91090c2ed75722ea21b5d3851039a015dc02255cf036b90623e79c1cfb91a744bbe7a36afa70469fdae3903d7a4b16fc4f5741eb41b879d3c1425a58eed3c705b,flickr30k:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F623329%2F1111749%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T095117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2f4c9268c4e9e90b8c31750a5d96f94119e505ce6277f588110d6294704fa3943e1ca739fbc82038705a77620116852df69083bb10154b192582dd2ba5e2e6eddf5f150791761a0f72432179cf683f289afc20f8398dd6f5098dce1766dc01a57e5838f821b319a6fda97055ce0666dd49ea69f43a177c97c2571fd8677ce41eaa90214f5c9e5c0fed0ed98d52d52fb66a17daaec96f4b09a0d3b30b370a49df75301cc822d24c61fa588c604117b00b5a5ba5bb903fdb4e21501146313d1b8cea47d097d8aada92366828c2d89c69e178336ba1d8e7a52ea35bbd1a3198f95fa0e957a9cccc1c641ca8f1d3d10e1523700eac5f00674442c4e812bc3131f7ae,coco-2017-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F857191%2F1462296%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T095117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D35e451c7fe6c5dac69c944bb48c309da04304c74fe57197a998a15211b7c85afc1e040390709548d2d5b9e7f6d02a9fec7fdbb704c46da0825225c422ed430a0fca996c7e4a1263ec6301af94bd31ea340399f354f9de7ef7208f5657d0df979afd5e2e45b19f2f1428536cb46da64a6b1d79544babc4a89aede2a7b323632f9228fa02b31d2890aaf59ce788a92dd83ea5009b0201f83d3a805a6277f3171aaa8df2a8553403de6e51e46236be980d5567a469a51699f1ef1b320ff69a08cb402b852f30f5bed6b24febf6594a04e4827a10ac0172e4e3fb48136bd10366f34d09077dbe667c18c14d2355f970aea11434252553af253b128b2011c8f35c174,images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4728196%2F8023465%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T095117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D621d8279b8f705fd0deab940bdf862fb609bf68200c1eab71802da563215426b950a294965b63f318b88390773ea3f0a7013945a4ecce3bc949cbffc2cdb216f2b7b44dc551cc6194904814f8f8a086524e3a279f48d97dce8076cc38bc78621820f09369fce5c9b4a88e731b5e60a704c38c837163adf8d7f96ae3253de7b59fe7d2509d732911a910d67681da981a2afa0a35901426991fea95910a05d1daa11e4f208cfe05ba620726efbe3dd1a257e17a9f76f79fc875a0b3d54bcd138f779b7fbd229b88f2659d3f0ec95290a5d5fc1cd8b6fa2755425b51b05ee7a7c3f5426657179b0b3ea1f690dc5cdd06a3e91952ff3f01f6e6d375e1e8800f2aa7f,testing:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4728207%2F8023481%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T095117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4353c89119bf0a897e10ec871c4e53b5d3fb93496da79f595e29cd2a9981d297dfdb10bb8c30a69f1f6b6ef713cacc9f6e7be4edbad8ccba31c01d8bc8807faaf1047b88a45d307667a67d455a728cfb651085fd1dd106959b56fd4c9ff187b5e42ab41233940e14526a1f536571e049d1f52b6433e08a06527a8562ab9ea20d141ee22b35a170a69c54861aa85cc8260ce18f8ae4dc1f2ec350811cb90bb42b2ce058b41188dc17e71b332633e3f252591d519d43a83ba78b61b43029c2456b94cd956c956dd32979c35975ef51601dcba27cd0ecd6f8f2e33ef901a07667ead716f644efaf02aeaab19016a4eefa902c29f54e510075cf6b2cfe1745a9bd70,testcode:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4920752%2F8284832%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T095117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D345e776661e2beb4a9e93e2287eb37bacec9cb20255b9dd5882e46742b45b35fc758318a08451cadaca01dec026aefc4b24523f8ad2b981f4d89a719bdc3580e5338949348ff0eff5670faa0686ccbb433483c1196d3f521d28086323f3c2e530b68e8beda92aceeb0f9e1d871245c705f9d6212a2abd5b04b7c4d0ceb305c26999eb6c8d28effef0d5a5f74f2cc881031822d3f677aefc58dc04be529a746f07aebd75267e19eb23cbd8f50f3b7e94e1838424bc1ee3156f9b8efabff671e4bb3e6f44877a4840b262aedf6deb5f846738db5b023af8794acf1f30e2d58f1a71f73fccfc6a29361b609800842a7bbc0a8ccd5b4d2810824401e791132b6a849,dataset-script:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4920953%2F8285287%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T095117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D37a202f50e91130aa049555d45d113439373f9d8509cbc9067c96e05704871b387d8360edd9d9bf875e5cc3cff0ff785719f5eff7670c03f064abc4552e862fff70fd97a36da7e38118c7c199efc47426215200d8214223a2180b2492d25a4cd12e0ef7bb93e1f82b17c8140c05477659c607451f7747e31baebd39c176b784c6f34cd9fd0302eaf8419c9d02ec2bb5a2aa192e542550e659a0d7755328a426856b5e336d1e6cd077234b2f02bc2a1da7702b79cea2c44af10cdcd251a34fb02046224ed691aaf0722643f973188b6562a44aa20145d05e06a4dbb8a04b2f6880e91f91f8f4a8676aaa07d5ef23458359a779fe9b8e9d613323e290759b1b181'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#Setup Gdrive file download extention
!conda install -y gdown

In [ ]:
!gdown --id 1_AP0E8BGgXwOJ9e9SpW9LYHzqKJEjZsn


In [ ]:
!unzip /kaggle/working/image-captioning-output-save.zip

In [ ]:
import os
os.remove('/kaggle/working/image-captioning-output-save.zip')

In [ ]:
!pip install datasets
!pip install accelerate
!pip install transformers

The Illustrated Image Captioning using transformers
Contents
Introduction
Vision Encoder Decoder Architecture
Initialize VisionEncoderDecoderModel
Data Loading and Preparation
Define seq2seq training arguments
Define metric
Training
Inference
References
Cited as IntroductionPermalink
Image captioning is the process of generating caption i.e. description from input image. It requires both Natural language processing as well as computer vision to generate the caption.

The popular benchmarking dataset which has images and its caption are:

Common Objects in Context (COCO). A collection of more than 120 thousand images with descriptions.
Flickr 8K: A collection of 8 thousand described images taken from flickr.com.
Flickr 30K: A collection of 30 thousand described images taken from flickr.com.
Try trained model: https://huggingface.co/nlpconnect/vit-gpt2-image-captioning

Image Vision Encoder Decoder ArchitecturePermalink Image

The Vision Encoder Decoder Model can be used to initialize an image-to-text model with any pre-trained Transformer-based vision model as the encoder (e.g. ViT, BEiT, DeiT, Swin) and any pre-trained language model as the decoder (e.g. RoBERTa, GPT2, BERT, DistilBERT).

Image captioning is an example, in which the encoder model is used to encode the image, after which an autoregressive language model i.e. the decoder model generates the caption.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
import datasets
from transformers import VisionEncoderDecoderModel, AutoFeatureExtractor,AutoTokenizer
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import nltk
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    nltk.download("punkt", quiet=True)

Initialize VisionEncoderDecoderModel

In [ ]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor, ViTImageProcessor

image_encoder_model = "google/vit-base-patch16-224-in21k"
#image_encoder_model = "google/vivit-b-16x2-kinetics400"
text_decode_model = "gpt2"

#model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(image_encoder_model, text_decode_model)

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

FeatureExtractor is used to extract features i.e. image patch resolution of 16x16.

Tokenizer is used to tokenize and encode text features.

In [ ]:
#from transformers import *
# image feature extractor
#feature_extractor = AutoFeatureExtractor.from_pretrained(image_encoder_model)
#feature_extractor =ViTImageProcessor.from_pretrained(image_encoder_model)
# text tokenizer
#tokenizer = AutoTokenizer.from_pretrained(text_decode_model)

In [ ]:
# GPT2 only has bos/eos tokens but not decoder_start/pad tokens
tokenizer.pad_token = tokenizer.eos_token

# update the model config
model.config.eos_token_id = tokenizer.eos_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
output_dir = "vit-gpt-model"
model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Data Loading and PreparationPermalink
We are going to use sample dataset from ydshieh/coco_dataset_script.

For using Full COCO dataset (2017), you need to download it manually first:

In [ ]:
#wget http://images.cocodataset.org/zips/train2017.zip
#wget http://images.cocodataset.org/zips/val2017.zip
#wget http://images.cocodataset.org/zips/test2017.zip
#wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
#wget http://images.cocodataset.org/annotations/image_info_test2017.zip Then to load the dataset:

#COCO_DIR = "/kaggle/input/coco-2017-dataset"
#ds = datasets.load_dataset("/kaggle/input/coco-2017-dataset","2017", data_dir=COCO_DIR)
#COCO_DIR = "/kaggle/input/coco-2017-dataset"

# Load the COCO2017 dataset using the correct path
#ds = datasets.load_dataset("coco", "2017", data_dir=COCO_DIR)

In [ ]:
import datasets
ds = datasets.load_dataset("ydshieh/coco_dataset_script", "2017", data_dir="./dummy_data/")
ds

In [ ]:
#import datasets
#ds = datasets.load_dataset("/kaggle/input/coco-2017-dataset/coco2017", "default", data_dir="./annotations", field='./annotations')
    #ds

In [ ]:
# print single example
ds['train'][1]

In [ ]:
from PIL import Image

# text preprocessing step
def tokenization_fn(captions, max_target_length):
    """Run tokenization on captions."""
    labels = tokenizer(captions,
                      padding="max_length",
                      max_length=max_target_length).input_ids

    return labels

# image preprocessing step
def feature_extraction_fn(image_paths, check_image=True):
    """
    Run feature extraction on images
    If `check_image` is `True`, the examples that fails during `Image.open()` will be caught and discarded.
    Otherwise, an exception will be thrown.
    """

    model_inputs = {}

    if check_image:
        images = []
        to_keep = []
        for image_file in image_paths:
            try:
                img = Image.open(image_file)
                images.append(img)
                to_keep.append(True)
            except Exception:
                to_keep.append(False)
    else:
        images = [Image.open(image_file) for image_file in image_paths]

    encoder_inputs = feature_extractor(images=images, return_tensors="np")

    return encoder_inputs.pixel_values

def preprocess_fn(examples, max_target_length, check_image = True):
    """Run tokenization + image feature extraction"""
    image_paths = examples['image_path']
    captions = examples['caption']

    model_inputs = {}
    # This contains image path column
    model_inputs['labels'] = tokenization_fn(captions, max_target_length)
    model_inputs['pixel_values'] = feature_extraction_fn(image_paths, check_image=check_image)

    return model_inputs

PIL for Flickr8k

In [ ]:
processed_dataset = ds.map(
    function=preprocess_fn,
    batched=True,
    fn_kwargs={"max_target_length": 128},
    remove_columns=ds['train'].column_names
)

In [ ]:
processed_dataset

Define seq2seq training argumentsPermalink

In [ ]:
#from transformers import TrainingArguments

#args = TrainingArguments("working_dir")
#args = args.set_evaluate(strategy="steps", steps=100)
#args.eval_steps

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    #save_strategy="epoch",
    #evaluation_strategy="epoch",
    #steps=1000,
    #save_steps=100,
    evaluation_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir="./image-captioning-Vit-GPT2-Flickr8k",
    overwrite_output_dir=True,
    #evaluation_strategy="steps",
    save_total_limit=3,
    load_best_model_at_end=True,
    #tpu_num_cores
    #accelerator_config (str, dict, or AcceleratorConfig, optional),
    push_to_hub=True,
    #resume_from_checkpoint=/kaggle/working/image-captioning-Vit-GPT2-Flickr8k/checkpoint-3000,
    #hub_token=,
    hub_always_push=True,
)

Define Metric

In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
import evaluate
metric = evaluate.load("rouge")

In [ ]:
import numpy as np

ignore_pad_token_for_loss = True


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
                                                     decoded_labels)

    result = metric.compute(predictions=decoded_preds,
                            references=decoded_labels,
                            use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

Training

In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['validation'],
    data_collator=default_data_collator,
)

In [ ]:
#trainer.train()

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load the CSV file containing image paths and captions
csv_file_path = "/kaggle/input/flickr8k/captions.txt"
df = pd.read_csv(csv_file_path)
df["image"] = "/kaggle/input/flickr8k/Images/" + df["image"]
# Limit df to 20,000 rows
#df = df.head(5000)
# Get the second part of the dataset (rows 10,000 to 20,000)
#second_part_df = df.iloc[10000:20000]
# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(df))
valid_size = int(0.1 * len(df))
test_size = len(df) - train_size - valid_size

train_df = df[:train_size]
valid_df = df[train_size:train_size + valid_size]
test_df = df[train_size + valid_size:]

# Convert each split into a Dataset object
train_data = Dataset.from_pandas(train_df)
valid_data = Dataset.from_pandas(valid_df)
test_data = Dataset.from_pandas(test_df)

# Organize the splits into a DatasetDict
ds = DatasetDict({
    "train": train_data,
    "validation": valid_data,
    "test": test_data
})

# Access the DatasetDict
print(ds)
test_data
test_df

In [ ]:
import torch
from PIL import Image
class ImageCapatioingDataset(torch.utils.data.Dataset):
    def __init__(self, ds, ds_type, max_target_length):
        self.ds = ds
        self.max_target_length = max_target_length
        self.ds_type = ds_type

    def __getitem__(self, idx):
        image_path = self.ds[self.ds_type]['image'][idx]
        caption = self.ds[self.ds_type]['caption'][idx]
        model_inputs = dict()
        model_inputs['labels'] = self.tokenization_fn(caption, self.max_target_length)
        model_inputs['pixel_values'] = self.feature_extraction_fn(image_path)
        return model_inputs

    def __len__(self):
        return len(self.ds[self.ds_type])

    # text preprocessing step
    def tokenization_fn(self, caption, max_target_length):
        """Run tokenization on caption."""
        labels = tokenizer(caption,
                          padding="max_length",
                          max_length=max_target_length).input_ids

        return labels

    # image preprocessing step
    def feature_extraction_fn(self, image_path):
        """
        Run feature extraction on images
        If `check_image` is `True`, the examples that fails during `Image.open()` will be caught and discarded.
        Otherwise, an exception will be thrown.
        """
        image = Image.open(image_path)

        encoder_inputs = feature_extractor(images=image, return_tensors="np")

        return encoder_inputs.pixel_values[0]


train_ds = ImageCapatioingDataset(ds, 'train', 64)
eval_ds = ImageCapatioingDataset(ds, 'validation', 64)


# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=default_data_collator,
)


In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./image-captioning-Vit-GPT2-Flickr8k")

In [ ]:
tokenizer.save_pretrained("./image-captioning-Vit-GPT2-Flickr8k")

Inference

In [ ]:
from transformers import pipeline

In [ ]:
# full dataset trained model can be found at https://huggingface.co/nlpconnect/vit-gpt2-image-captioning
image_captioner = pipeline("image-to-text", model="./image-captioning-Vit-GPT2-Flickr8k")

In [ ]:
image_captioner("/kaggle/input/testing/free-images.jpg")

In [ ]:
image_captioner("/kaggle/input/testing/nature-image-for-website.jpg")

In [ ]:
image_captioner("/kaggle/input/testing/Polish__.jpg")

In [ ]:
!zip -r image-captioning-output-save.zip /kaggle/working/image-captioning-Vit-GPT2-Flickr8k


## from IPython.display import FileLink
FileLink(r'image-captioning-output-save.zip')
